# Imports

In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import geopandas as gpd
import bs4

path = '../src/'
sys.path.append(path)

# Reading the data

In [ ]:
d_translate = {
"Andalusia": "Andalucia",
"Aragon": "Aragon",
"Asturias": "Asturias",
"Balearic Islands": "Baleares",
"Basque Country": "Pais Vasco",
"Canary Islands": "Canarias",
"Cantabria": "Cantabria",
"Castile and Leon": "Castilla-Leon",
"Castile-La Mancha": "Castilla-La Mancha",
"Catalonia": "Cataluña",
"Ceuta": "Ceuta",
"Comunidad Valenciana": "Valencia",
"Extremadura": "Extremadura",
"Galicia": "Galicia",
"La Rioja": "La Rioja",
"Madrid": "Madrid",
"Melilla": "Melilla",
"Murcia": "Murcia",
"Navarre": "Navarra",
}

In [ ]:
d_sheets = {
   "Table I.B2.1":"Mathematics performance",
   "Table I.B2.2":"Reading performance",
   "Table I.B2.3":"Science performance",
}
d_data ={}

for k,v in d_sheets.items():
   df = pd.read_excel('../data/pisa2022-en.xlsx', sheet_name=k, 
                   nrows=19,
                   skiprows=30,
                   usecols="A,B,D")
   measure = f'mean_{v.split(" ")[0].lower()}'
   std = f'std_{v.split(" ")[0].lower()}'

   df.columns=['region', measure, std]
   df[measure] = df[measure].round(1)
   df[std] = df[std].round(1)
   df['region'] = df['region'].str.strip()
   df['region'] = df['region'].replace(d_translate)
   d_data[v] = df
   print(df.columns)
   

# Putting together the dats with geojson with Spain communitie information

In [ ]:
gdf = gpd.read_file(
"https://raw.githubusercontent.com/codeforgermany/click_that_hood/main/public/data/spain-communities.geojson",
crs="epsg:4326"
)
gdf = gdf.rename(columns={'name':'region'})

In [ ]:
for k,v in d_data.items():
    gdf = gdf.join(v.set_index("region"), 
               on="region",
               how='inner')

In [ ]:
l_coofficial  =  ['Cataluña',
 'Baleares',
 'Valencia',
 'Navarra',
 'Galicia',
 'Pais Vasco']


In [ ]:
gdf['mean_f_score'] = ((gdf['mean_mathematics'] + gdf['mean_reading'] + gdf['mean_science'])/3.0).round(1)
gdf['std_f_score'] = gdf.apply(lambda row:
                            np.sqrt(row['std_mathematics']**2 +
                                     row['std_reading']**2 + 
                                     row['std_science']**2),
                                     axis = 1
                                     ).round(1)
gdf['n_languages'] = gdf['region'].apply(lambda x: 2 if x in l_coofficial else 1)

In [ ]:
gdf.groupby('n_languages')[['mean_mathematics',
                            'mean_reading',
                            'mean_science',
                            'mean_f_score'
                            ]].mean()

My initial hypothesis was that having more than one official language would decrease the reading score. This data seems to be a good starting point to test this hypothesis.

In [ ]:
import seaborn as sns

In [ ]:
sns.barplot(data=gdf.sort_values('mean_f_score'), x='region', y='mean_f_score')
plt.xticks(rotation=90)


# Merging with Population data
Let's add the population to obtain weight average of scores instead of simple average.

In [ ]:
import requests
from bs4 import BeautifulSoup
url = "https://es.wikipedia.org/wiki/Anexo:Comunidades_y_ciudades_aut%C3%B3nomas_de_Espa%C3%B1a"

In [ ]:

# Read the tables from the Wikipedia page
tables = pd.read_html(url)

# Select the desired table
table_index = 0  # Index of the table you want to read
df = tables[table_index]

Ceuta and Melilla are indeed the worst performing regions, let's remove then as outliers.

In [ ]:
#remove accents
df['Nombre'] = df['Nombre'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df = df.rename(columns={'Nombre':'region'})

In [ ]:
d_translate = {"Comunidad Valenciana": "Valencia",
               "Principado de Asturias": "Asturias",
               "Comunidad de Madrid": "Madrid",
                "Cataluna": "Cataluña",
               "Islas Baleares": "Baleares",
               "Castilla y Leon": "Castilla-Leon",
               "Region de Murcia": "Murcia",
               "Comunidad Foral de Navarra": "Navarra",}

In [ ]:
df['region'] = df['region'].replace(d_translate)


In [ ]:
gdf = gdf.merge(df, on='region', how='left')

In [ ]:
#gdf = gdf[gdf['region'].apply(lambda x: x not in ['Ceuta','Melilla'])]


We need the column "Porcentaje Poblacion" but we need to make sure we can convert it to float.

In [ ]:
gdf['weight'] = gdf['Porcentaje población'].str.replace('%','').str.replace(',','.').astype(float)/100.0

In [ ]:
gdf.groupby('n_languages')['weight'].sum().reset_index()

In [ ]:
gdf['mean_f_score_weighted'] = gdf['mean_f_score']*gdf['weight']
gdf['mean_reading_weighted'] = gdf['mean_reading']*gdf['weight']
scores_per_language = gdf.groupby('n_languages')[['mean_reading_weighted','mean_f_score_weighted','weight']].sum()

In [ ]:
scores_per_language

In [ ]:
scores_per_language = scores_per_language.reset_index()
scores_per_language['mean_reading_weighted'] = scores_per_language['mean_reading_weighted']/scores_per_language['weight']
scores_per_language['mean_f_score_weighted'] = scores_per_language['mean_f_score_weighted']/scores_per_language['weight']

In [ ]:
scores_per_language

In [ ]:
(
    scores_per_language['mean_reading_weighted'].iloc[0]-
    scores_per_language['mean_reading_weighted'].iloc[1]
    )/scores_per_language['mean_reading_weighted'].iloc[0]*100

With 1.1% of diffference in scores I can say that the difference in scores between regions with one and two official languages is not statistically significant.

My hypothesis was wrong!

# Geographical Plot

continuous colors:
aggrnyl     agsunset    blackbody   bluered     blues       blugrn      bluyl       brwnyl
bugn        bupu        burg        burgyl      cividis     darkmint    electric    emrld
gnbu        greens      greys       hot         inferno     jet         magenta     magma
mint        orrd        oranges     oryel       peach       pinkyl      plasma      plotly3
pubu        pubugn      purd        purp        purples     purpor      rainbow     rdbu
rdpu        redor       reds        sunset      sunsetdark  teal        tealgrn     turbo
viridis     ylgn        ylgnbu      ylorbr      ylorrd      algae       amp         deep
dense       gray        haline      ice         matter      solar       speed       tempo
thermal     turbid      armyrose    brbg        earth       fall        geyser      prgn
piyg        picnic      portland    puor        rdgy        rdylbu      rdylgn      spectral
tealrose    temps       tropic      balance     curl        delta       oxy         edge
hsv         icefire     phase       twilight    mrybm       mygbm

In [ ]:

px.choropleth_mapbox(
    gdf, 
    geojson=gdf["geometry"].__geo_interface__, 
    locations=gdf.index, 
    color="mean_reading", 
    color_continuous_scale="tropic",
    hover_name="region",
    hover_data=["mean_f_score","n_languages",
                "mean_mathematics","mean_reading","mean_science"],
    labels = {'region': 'region',
              "n_languages":"Num. of off. lang.",
              'mean_f_score':'Mean PISA score',
              'mean_mathematics':'Mean math score',
              'mean_reading':'Mean reading score',
              'mean_science':'Mean science score'
              },
    title='Mean PISA score by region',
).update_layout(
    mapbox={
        "style": "carto-positron",
        "center": {
            "lon": -3.640774937362015,
            "lat": 39.764566572736605,
        },
        "zoom":4
    },
    margin={"l":0,"r":0,"t":0,"b":0}
)